# Import Libs

In [6]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors


import warnings
warnings.filterwarnings("ignore")

In [7]:
orders = pd.read_csv('./S_Data/orders.csv')
products = pd.read_csv('./S_Data/products.csv')
product_categories = pd.read_csv('./S_Data/product_category_name_translation.csv')
order_items = pd.read_csv('./S_Data/order_items.csv')

<img src="https://github.com/senolcemhan98/templates/blob/main/OlistTables.png?raw=true" />

# Strategy 

- build ML and TS model for regular products
- for new products have a look the nearest neighbor in the same category to match and assign the nearest neighbor's demand for new product

In [8]:
merged_data = order_items.merge(orders, on = 'order_id', how='left')
# Get the maximum date in the dataset
merged_data['order_purchase_timestamp'] = pd.to_datetime(merged_data['order_purchase_timestamp'])
max_date = merged_data['order_purchase_timestamp'].max()

# Calculate the date 7 days ago from the maximum date
seven_days_ago = max_date - pd.Timedelta(days=7)

# Filter the DataFrame to include only the sales data from the last 7 days
last_7_days = merged_data[merged_data['order_purchase_timestamp'] >= seven_days_ago]

other_days = merged_data[merged_data['order_purchase_timestamp'] < seven_days_ago]

# new products which has no seen in before 7 days
new_product_list = list(set(last_7_days['product_id'].unique())-set(other_days['product_id'].unique()))

new_product_data = merged_data[merged_data['product_id'].isin(new_product_list)]
regular_product_data = merged_data[merged_data['product_id'].isin(new_product_list) == False]

In [9]:
# Split into train and test
train_data = regular_product_data[regular_product_data['order_purchase_timestamp'] < seven_days_ago]
test_data = regular_product_data[regular_product_data['order_purchase_timestamp'] >= seven_days_ago]

# Aggregate data to daily level
train_data_daily = train_data.groupby(['product_id', train_data['order_purchase_timestamp'].dt.date]).agg({'order_item_id': 'count'}).reset_index()
train_data_daily.columns = ['product_id', 'date', 'demand']

test_data_daily = test_data.groupby(['product_id', test_data['order_purchase_timestamp'].dt.date]).agg({'order_item_id': 'count'}).reset_index()
test_data_daily.columns = ['product_id', 'date', 'demand']

# Preparing the data for model training
X_train = train_data_daily[['product_id', 'date']]
y_train = train_data_daily['demand']

X_test = test_data_daily[['product_id', 'date']]
y_test = test_data_daily['demand']

# Feature Engineering: converting date to ordinal for simplicity
X_train['date'] = pd.to_datetime(X_train['date']).map(pd.Timestamp.toordinal)
X_test['date'] = pd.to_datetime(X_test['date']).map(pd.Timestamp.toordinal)


# ML Method

In [10]:
# Define column transformer to handle categorical product_id
preprocessor = ColumnTransformer(
    transformers=[
        ('product_id', OneHotEncoder(), ['product_id']),
        ('date', 'passthrough', ['date'])
    ]
)

# Create pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Fit the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Calculate the mean squared error for evaluation
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Adding product_id to predictions for further analysis
predictions_df = X_test.copy()
predictions_df['predicted_demand'] = predictions

# Convert 'date' from ordinal back to datetime for display purposes
predictions_df['date'] = predictions_df['date'].apply(dt.datetime.fromordinal)


predictions_df.head()

Mean Squared Error: 0.457379222080865


,product_id,date,predicted_demand
0,01a5038835acf17f141c7247956b72d6,2018-08-28,0.997780
1,02b750ee6f6f12a23c311b775ee4427a,2018-08-28,0.987067
2,060c17562f97e5bb60bc0dfa4dd5b3f2,2018-08-27,1.276978
3,06601c3059e35a3bf65e72f2fd2ac626,2018-08-29,1.337344
4,06bf70b6e1d67d96308235ef350edc61,2018-08-27,1.045959


In [11]:
predictions_df['date'] = predictions_df['date'].dt.date.astype('O')
pred_test_data  = predictions_df.merge(test_data_daily,on=['product_id','date'],how='left')
pred_test_data[pred_test_data['demand']>=2].head()

,product_id,date,predicted_demand,demand
15,2b4472df15512a2825ae86fd9ae79335,2018-08-29,1.003350,2
23,4a545fda8038a360bd93a5f1ac419a4e,2018-08-27,1.003552,3
26,59536eeb13a52ef81966f81b91fb481e,2018-08-28,1.972566,3
34,73326828aa5efe1ba096223de496f596,2018-08-28,2.445821,2
40,8d4dac6177fb8134f26fb4c5cc6c0aff,2018-08-29,1.283599,5


In [12]:
# I want to check the product which is product_id = 8d4dac6177fb8134f26fb4c5cc6c0aff if the daily demand generally around 5?
train_data_daily[train_data_daily['product_id']=='8d4dac6177fb8134f26fb4c5cc6c0aff'].sort_values('date',ascending=False)
#looks like an anomaly 

,product_id,date,demand
52616,8d4dac6177fb8134f26fb4c5cc6c0aff,2018-07-03,1
52615,8d4dac6177fb8134f26fb4c5cc6c0aff,2018-06-19,1
52614,8d4dac6177fb8134f26fb4c5cc6c0aff,2018-06-18,2
52613,8d4dac6177fb8134f26fb4c5cc6c0aff,2018-06-08,2
52612,8d4dac6177fb8134f26fb4c5cc6c0aff,2018-05-20,1
52611,8d4dac6177fb8134f26fb4c5cc6c0aff,2018-05-18,1
52610,8d4dac6177fb8134f26fb4c5cc6c0aff,2018-05-08,1


for New products
- my idea of here to match the new products with nearest neighbor product in same category. and assume like they're similar product and have similar demand. 
<br>
<hr>
Note: if new product category has no sales history. My approach won't work :( 
<hr>

In [13]:
new_products = products[products['product_id'].isin(new_product_data['product_id'].unique())]
regular_products = products[(products['product_id'].isin(new_product_data['product_id'].unique())==False) & (products['product_id'].isin(predictions_df['product_id'].unique()))]
# regular_products : if has sales and not a new prod

# Columns to consider for nearest neighbor matching
numerical_columns = ["product_weight_g", "product_length_cm", "product_height_cm", "product_width_cm"]

# Initialize a list to store results
results = []

# Iterate through each unique category
for category in new_products['product_category_name'].unique():
    # print(category)
    # Filter existing products and new products by category
    category_existing = regular_products[regular_products['product_category_name'] == category]
    category_new = new_products[new_products['product_category_name'] == category]

    # Extract the numerical columns
    X = category_existing[numerical_columns]
    new_X = category_new[numerical_columns]

    try:
        # Normalize the data
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        new_X_scaled = scaler.transform(new_X)
    

        # Fit the Nearest Neighbors model
        nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(X_scaled)
        distances, indices = nbrs.kneighbors(new_X_scaled)

        # Prepare results for the current category
        category_new['nearest_neighbor_id'] = category_existing.iloc[indices.flatten()]['product_id'].values
        category_new['distance_to_nearest'] = distances.flatten()
        results.append(category_new)

    except:
        print(f'no sales history for - {category}  ')

# Combine results for all categories
matched_products = pd.concat(results, ignore_index=True)

matched_products

no sales history for - construcao_ferramentas_jardim  
no sales history for - artigos_de_festas  
no sales history for - portateis_cozinha_e_preparadores_de_alimentos  
no sales history for - malas_acessorios  
no sales history for - ferramentas_jardim  
no sales history for - papelaria  


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,nearest_neighbor_id,distance_to_nearest
0,6f4ed02e363461e27168f72bc33ffb80,cama_mesa_banho,59.0,344.0,3.0,12000.0,60.0,23.0,54.0,981f6a96c09b7d28e5c5f2fbb282fcde,19.228282
1,0896208dd8490b3dbb75344a92ab4106,cama_mesa_banho,56.0,1023.0,1.0,1350.0,36.0,20.0,36.0,b74d57e3377dd78aace6d070fc48cc88,1.390915
2,04008f5a086abe248a3c76bc2fa4f4e1,cama_mesa_banho,41.0,2201.0,1.0,1250.0,48.0,14.0,68.0,981f6a96c09b7d28e5c5f2fbb282fcde,3.691373
3,49d23d4887468040921b413a6d4f3ec7,cama_mesa_banho,52.0,1072.0,1.0,1350.0,39.0,7.0,31.0,01a5038835acf17f141c7247956b72d6,0.512560
4,e84d1a4a08db13e8dea6d400a1c6a3f0,alimentos,34.0,926.0,2.0,700.0,24.0,15.0,24.0,73326828aa5efe1ba096223de496f596,6.007798
5,896496916bfd78848143a60e492dbcf8,alimentos,35.0,567.0,4.0,225.0,21.0,9.0,14.0,ab1106e227a745836a0dc83b374e2575,1.325341
6,7001d71d1ad858e07e5a341649412e76,utilidades_domesticas,52.0,933.0,2.0,550.0,30.0,16.0,25.0,f8e4c6a9bf1c1277191c445474b61778,0.383655
7,fafa1432833bfab02d8039194021bfc9,utilidades_domesticas,57.0,326.0,1.0,800.0,16.0,2.0,11.0,59536eeb13a52ef81966f81b91fb481e,1.316567
8,3207053643b68f43eed19750c0ad8157,utilidades_domesticas,46.0,1900.0,2.0,500.0,33.0,12.0,18.0,f8e4c6a9bf1c1277191c445474b61778,1.131843
9,d6897f1dfdc99bbbdfda785158793d45,utilidades_domesticas,60.0,1064.0,1.0,390.0,24.0,20.0,24.0,e31c5933157c3e4134a0f536f7bcc14e,0.487976


In [14]:
pred_new_prods = matched_products.merge(predictions_df, left_on='nearest_neighbor_id',right_on='product_id',how='left')

In [15]:
pred_new_prods[['product_id_x', 'date',	'predicted_demand']]

,product_id_x,date,predicted_demand
0,6f4ed02e363461e27168f72bc33ffb80,2018-08-27,1.007688
1,0896208dd8490b3dbb75344a92ab4106,2018-08-27,1.046196
2,04008f5a086abe248a3c76bc2fa4f4e1,2018-08-27,1.007688
3,49d23d4887468040921b413a6d4f3ec7,2018-08-28,0.997780
4,e84d1a4a08db13e8dea6d400a1c6a3f0,2018-08-27,2.445829
5,e84d1a4a08db13e8dea6d400a1c6a3f0,2018-08-28,2.445821
6,896496916bfd78848143a60e492dbcf8,2018-08-27,1.382020
7,7001d71d1ad858e07e5a341649412e76,2018-08-27,1.003199
8,fafa1432833bfab02d8039194021bfc9,2018-08-28,1.972566
9,3207053643b68f43eed19750c0ad8157,2018-08-27,1.003199


# Sarimax Model

In [70]:
import pandas as pd
from prophet import Prophet

# Step 1: Load data
orders = pd.read_csv('./S_Data/orders.csv')
order_items = pd.read_csv('./S_Data/order_items.csv')

# Step 2: Merge relevant columns
merged_df = pd.merge(orders[['order_id', 'order_purchase_timestamp']], order_items[['order_id', 'product_id']], on='order_id')

# Convert purchase timestamp to datetime
merged_df['order_purchase_timestamp'] = pd.to_datetime(merged_df['order_purchase_timestamp'])

# Step 3: Aggregate data by product_id and date
daily_demand = merged_df.groupby(['product_id', pd.Grouper(key='order_purchase_timestamp', freq='D')]).size().reset_index(name='demand')

# Step 4: Train a time series model for each product_id
product_ids = daily_demand['product_id'].unique()
forecast_data = pd.DataFrame()
daily_demand

,product_id,order_purchase_timestamp,demand
0,00066f42aeeb9f3007548bb9d3f33c38,2018-05-20,1
1,00088930e925c41fd95ebfe695fd2655,2017-12-12,1
2,0009406fd7479715e4bef61dd91f2462,2017-12-21,1
3,000b8f95fcb9e0096488278317764d19,2018-08-01,1
4,000b8f95fcb9e0096488278317764d19,2018-08-10,1
...,...,...,...
94586,fffdb2d0ec8d6a61f0a0a0db3f25b441,2018-04-19,1
94587,fffdb2d0ec8d6a61f0a0a0db3f25b441,2018-05-01,1
94588,fffdb2d0ec8d6a61f0a0a0db3f25b441,2018-08-07,1
94589,fffdb2d0ec8d6a61f0a0a0db3f25b441,2018-08-10,1


In [72]:
import pandas as pd
from prophet import Prophet

# Step 1: Load data
orders = pd.read_csv('./S_Data/orders.csv')
order_items = pd.read_csv('./S_Data/order_items.csv')

# Step 2: Merge relevant columns
merged_df = pd.merge(orders[['order_id', 'order_purchase_timestamp']], order_items[['order_id', 'product_id']], on='order_id')

# Convert purchase timestamp to datetime
merged_df['order_purchase_timestamp'] = pd.to_datetime(merged_df['order_purchase_timestamp'])

# Step 3: Aggregate data by product_id and date
daily_demand = merged_df.groupby(['product_id', pd.Grouper(key='order_purchase_timestamp', freq='D')]).size().reset_index(name='demand')

# Step 4: Train a time series model for each product_id
product_ids = daily_demand['product_id'].unique()
forecast_data = pd.DataFrame()

for product_id in product_ids:
    # Filter data for the current product_id
    product_data = daily_demand[daily_demand['product_id'] == product_id]
    
    # Rename columns for Prophet
    product_data.rename(columns={'order_purchase_timestamp': 'ds', 'demand': 'y'}, inplace=True)
    
    # Initialize Prophet model
    model = Prophet()
    
    # Check if the product has less than 2 data points
    if len(product_data) < 2:
        print(f"Not enough data for product_id: {product_id}. Forecasting as 1.")
        # Create a forecast with constant value 1
        forecast = pd.DataFrame({'ds': pd.date_range(start=product_data['ds'].max(), periods=7, freq='D'), 'yhat': 1})
    else:
        # Fit the model
        model.fit(product_data)
        
        # Make future dataframe for 7 days forecast
        future = model.make_future_dataframe(periods=7)
        
        # Forecast
        forecast = model.predict(future)
    
    # Add product_id column
    forecast['product_id'] = product_id
    
    # Concatenate forecast to forecast_data
    forecast_data = pd.concat([forecast_data, forecast], ignore_index=True)

# Output the forecasts for the next 7 days for each product_id
print(forecast_data[['ds', 'product_id', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7))


Not enough data for product_id: 00066f42aeeb9f3007548bb9d3f33c38. Forecasting as 1.
Not enough data for product_id: 00088930e925c41fd95ebfe695fd2655. Forecasting as 1.
Not enough data for product_id: 0009406fd7479715e4bef61dd91f2462. Forecasting as 1.
Not enough data for product_id: 000d9be29b5207b54e86aa1b1ac54872. Forecasting as 1.
Not enough data for product_id: 0011c512eb256aa0dbbb544d8dffcf6e. Forecasting as 1.
Not enough data for product_id: 00126f27c813603687e6ce486d909d01. Forecasting as 1.


02:56:06 - cmdstanpy - INFO - Chain [1] start processing
02:56:06 - cmdstanpy - INFO - Chain [1] done processing


Not enough data for product_id: 001b237c0e9bb435f2e54071129237e9. Forecasting as 1.


02:56:06 - cmdstanpy - INFO - Chain [1] start processing
02:56:06 - cmdstanpy - INFO - Chain [1] done processing


Not enough data for product_id: 001c5d71ac6ad696d22315953758fa04. Forecasting as 1.


...

Not enough data for product_id: 2a58eaba034398b020fe690c6dbb4b92. Forecasting as 1.
Not enough data for product_id: 2a592642399b33baf3a5729a6335b21c. Forecasting as 1.
Not enough data for product_id: 2a5ce93df0dd4ec1b23ebc2e599d793e. Forecasting as 1.
Not enough data for product_id: 2a5e8a686fea7f74cb246f729a53e650. Forecasting as 1.
Not enough data for product_id: 2a5fa7b501557e971bf63ec336f67fa2. Forecasting as 1.
Not enough data for product_id: 2a5fd02982127e736ae8f4d6d020fcc8. Forecasting as 1.
Not enough data for product_id: 2a61ae7c276ed675a0289aee54e2c9b0. Forecasting as 1.
Not enough data for product_id: 2a65f5c6c203f2e5079da9ac55814c61. Forecasting as 1.
Not enough data for product_id: 2a66698f3c9f8708a0505b5ab2cb380d. Forecasting as 1.
Not enough data for product_id: 2a6949b0a5f63681fccef03f5c628edd. Forecasting as 1.
Not enough data for product_id: 2a699d35a87e92a01747297243bfc987. Forecasting as 1.


03:36:09 - cmdstanpy - INFO - Chain [1] start processing
03:36:09 - cmdstanpy - INFO - Chain [1] done processing


Not enough data for product_id: 2a714dc40b27bbd0887215ed0cdc3bbe. Forecasting as 1.
Not enough data for product_id: 2a763f02c6a7a8ab7c1be0460ff891d0. Forecasting as 1.
Not enough data for product_id: 2a7835ac39ba7753ece75242d290f044. Forecasting as 1.
Not enough data for product_id: 2a7906396f06c254b7b32a8ed3e45894. Forecasting as 1.
Not enough data for product_id: 2a7d1be78b464ce799d177a408e327c0. Forecasting as 1.
Not enough data for product_id: 2a7e9f98e73a335bbb0ba2253d0c9b40. Forecasting as 1.
Not enough data for product_id: 2a8181f10547f2766c59997bbf9af643. Forecasting as 1.
Not enough data for product_id: 2a88b20e4ce366edc5efac1c5764d808. Forecasting as 1.
Not enough data for product_id: 2a8c2836c76d1810ac79e84bbdeff3a3. Forecasting as 1.
Not enough data for product_id: 2a962e2b5e4955e5ed284f1cf46b5296. Forecasting as 1.
Not enough data for product_id: 2a9b9ab4785e67b8ff4f1e6fcba1de7e. Forecasting as 1.
Not enough data for product_id: 2a9e3d335bbb23ffee8c9da6cecb7bb8. Forecastin

03:36:14 - cmdstanpy - INFO - Chain [1] start processing
03:36:17 - cmdstanpy - INFO - Chain [1] done processing


Not enough data for product_id: 2aaaa41fb5edc85f4814c73106f27d64. Forecasting as 1.
Not enough data for product_id: 2aae44062520332404c7c026b7cb03c1. Forecasting as 1.
Not enough data for product_id: 2aafde3b2df26664216fa72787b7dda0. Forecasting as 1.
Not enough data for product_id: 2ab31f78e069ddb51930c6ace3759e45. Forecasting as 1.
Not enough data for product_id: 2ab38f02a646dc3972785867d5234306. Forecasting as 1.
Not enough data for product_id: 2ab3937e84ae4f7c4b6549b05c95df0b. Forecasting as 1.


In [ ]:
forecast_data.to_csv('forecasts.csv')

In [5]:
forecast_data = pd.read_csv('forecasts.csv')
forecast_data[['ds', 'product_id', 'yhat']].head()

,ds,product_id,yhat
0,2018-05-20,00066f42aeeb9f3007548bb9d3f33c38,1.0
1,2018-05-21,00066f42aeeb9f3007548bb9d3f33c38,1.0
2,2018-05-22,00066f42aeeb9f3007548bb9d3f33c38,1.0
3,2018-05-23,00066f42aeeb9f3007548bb9d3f33c38,1.0
4,2018-05-24,00066f42aeeb9f3007548bb9d3f33c38,1.0
